In [11]:
# Advanced_RAG_Report_Generation.ipynb

# ## 🚀 고급 RAG 리포트 생성 파이프라인 (feat. LangGraph & Multi-Query)
#
# 이 노트북은 정형 데이터(Feature)와 모델 예측 결과를 입력받아,
# 실시간 웹 검색 및 다중 쿼리 검색(Multi-Query Retrieval)을 통해
# 전문가 수준의 심층 분석 리포트를 생성하는 LangGraph 파이프라인을 구현합니다.

# ### 0. 필수 라이브러리 설치
#
# # 만약 'googlesearch-python' 라이브러리가 설치되지 않았다면 아래 주석을 해제하고 실행해주세요.
# # !pip install langchain langgraph langchain_openai faiss-cpu beautifulsoup4 python-dotenv googlesearch-python

import os
import json
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv

# LangChain 관련 import
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever

# LangGraph 관련 import
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# --- 환경 설정 ---
# .env 파일에서 OpenAI API 키를 로드합니다.
load_dotenv()

# ### 1. 입력 데이터 정의
#
# `app.py`의 `parse_register_info_detailed`와 예측 모델을 통해 최종적으로 생성된 데이터를 입력으로 사용합니다.

initial_input = {
    "features": {
        '건축물_유형': '아파트',
        '근저당권_개수': 1,
        '채권최고액': 600000000,
        '신탁_등기여부': False,
        '압류_가압류_개수': 1,
        '선순위_채권_존재여부': True,
        '전입_가능여부': True,
        '우선변제권_여부': True,
        '주소': '서울특별시 강남구 테헤란로 123',
        '과거_전세가율': "81.50%"
    },
    "model_prediction": {
        "prediction": "주의",
        "risk_probability": "36.59%",
        "risk_score": "0.2087"
    }
}

print("✅ 1단계: 입력 데이터 준비 완료")
print(json.dumps(initial_input, indent=2, ensure_ascii=False))


# ### 2. RAG 파이프라인 구축 (웹 검색 -> DB 구축 -> Multi-Query Retriever)

# #### 2-1. 동적 웹 검색 및 데이터 로딩 (Web Search & Loading)

def generate_search_queries(data: dict) -> List[str]:
    """입력 데이터 기반으로 검색 쿼리 생성"""
    queries = ["전세 계약시 주의사항 최신"]
    features = data['features']
    if features.get('근저당권_개수', 0) > 0:
        queries.append("등기부등본 근저당권 위험성")
    if features.get('압류_가압류_개수', 0) > 0:
        queries.append("가압류 있는 집 전세 계약")
    if features.get('신탁_등기여부'):
        queries.append("신탁등기 부동산 임대차 계약")
    if float(features.get('과거_전세가율', '0%').strip('%')) > 80:
        queries.append("전세가율 높은 빌라 위험성")
    return list(set(queries))

# 구글 검색을 위한 함수 (라이브러리 API 변경에 따라 수정됨)
try:
    # 'googlesearch-python' 라이브러리는 'google'이라는 이름으로 import 될 수 있습니다.
    from googlesearch import search
except ImportError:
    print("'google' 모듈을 찾을 수 없습니다. 'pip install googlesearch-python' 명령어로 설치해주세요.")

def search_google(query, num_results=3):
    """구글 검색으로 URL 리스트 반환 (수정된 버전)"""
    urls = []
    # 최신 라이브러리 버전에 맞는 인자 사용 (num_results, lang)
    # tld, num, stop 인자는 현재 버전에서 TypeError를 유발할 수 있습니다.
    try:
        for url in search(query, num_results=num_results, lang='ko'):
            urls.append(url)
    except Exception as e:
        print(f"구글 검색 중 오류 발생: {e}")
    return urls


# 웹 페이지 로딩 및 Vector DB 구축
def create_vector_db_from_web(data: dict):
    """웹 검색 -> 문서 로딩 -> Vector DB 생성"""
    # 1. 검색 쿼리 생성
    queries = generate_search_queries(data)
    print(f"👉 생성된 검색 쿼리: {queries}")

    # 2. URL 수집
    all_urls = []
    for query in queries:
        urls = search_google(query)
        all_urls.extend(urls)
    unique_urls = list(set(all_urls))
    print(f"👉 수집된 URL ({len(unique_urls)}개): {unique_urls[:3]}...")

    if not unique_urls:
        print("⚠️ 검색된 URL이 없어 RAG 프로세스를 중단합니다.")
        return None

    # 3. WebBaseLoader로 문서 로딩
    loader = WebBaseLoader(
        unique_urls,
        fetch_options={"headers": {"User-Agent": "Mozilla/5.0"}},
        on_error=lambda e: print(f"URL 로드 에러: {e}")
    )
    docs = loader.load()

    # 4. 텍스트 분할 및 Vector DB 생성
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

# Vector DB 생성 실행
retriever = create_vector_db_from_web(initial_input)
if retriever:
    print("✅ 2단계: 실시간 웹 정보 기반 Vector DB 구축 완료")
else:
    print("❌ 2단계: Vector DB 구축 실패.")

# #### 2-2. Multi-Query Retriever 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)

if retriever:
    retriever_from_llm = MultiQueryRetriever.from_llm(
        retriever=retriever, llm=llm
    )
    print("✅ 2단계: Multi-Query Retriever 설정 완료")
else:
    retriever_from_llm = None
    print("❌ 2단계: Retriever가 없어 Multi-Query Retriever를 설정할 수 없습니다.")


# ### 3. LangGraph 워크플로우 설계

# #### 3-1. 그래프 상태(State) 정의

class ReportState(TypedDict):
    input_data: dict
    context_query: str
    retrieved_context: List[str]
    initial_analysis: str
    final_report: str

# #### 3-2. 그래프 노드(Node) 함수 정의

def generate_context_query(state: ReportState) -> ReportState:
    print("--- 1. RAG 검색용 질문 생성 노드 ---")
    input_data = state['input_data']
    prompt = ChatPromptTemplate.from_template("아래 데이터의 핵심 위험 요소를 요약하여 관련 법률 및 시장 정보를 검색하기 위한 질문을 만들어주세요.\n\n데이터: {data}")
    chain = prompt | llm | StrOutputParser()
    context_query = chain.invoke({"data": json.dumps(input_data, ensure_ascii=False)})
    print(f"👉 생성된 RAG 질문: {context_query}")
    return {"context_query": context_query}

def retrieve_context(state: ReportState) -> ReportState:
    print("--- 2. 컨텍스트 정보 검색 노드 ---")
    if not retriever_from_llm:
        print("⚠️ Retriever가 없어 컨텍스트 검색을 건너뜁니다.")
        return {"retrieved_context": ["참고 자료를 찾을 수 없습니다."]}
    context_query = state['context_query']
    docs = retriever_from_llm.invoke(context_query)
    retrieved_context = [doc.page_content for doc in docs]
    print(f"👉 {len(retrieved_context)}개의 관련 정보 조각 검색 완료")
    return {"retrieved_context": retrieved_context}

def generate_initial_analysis(state: ReportState) -> ReportState:
    print("--- 3. 1차 분석 초안 생성 노드 ---")
    prompt = ChatPromptTemplate.from_template(
        """
        당신은 부동산 AI 애널리스트입니다. 아래 원본 데이터와 참고 자료를 바탕으로 각 위험 요소에 대한 상세한 1차 분석을 작성해주세요.
        참고 자료를 근거로 제시하며 신뢰도를 높여주세요.

        ### 원본 데이터
        {data}

        ### 관련 참고 자료
        {context}
        """
    )
    chain = prompt | llm | StrOutputParser()
    initial_analysis = chain.invoke({
        "data": json.dumps(state['input_data'], ensure_ascii=False),
        "context": "\n---\n".join(state['retrieved_context'])
    })
    return {"initial_analysis": initial_analysis}

def generate_final_report(state: ReportState) -> ReportState:
    print("--- 4. 최종 리포트 생성 노드 ---")
    prediction = state['input_data']['model_prediction']['prediction']
    prompt = ChatPromptTemplate.from_template(
        """
        당신은 전문 부동산 컨설턴트입니다. 아래 내부 분석 자료를 바탕으로, 일반인 고객이 이해하기 쉬운 최종 보고서를 작성해주세요.
        최종 위험 등급 '{prediction}'을 강조하며, 아래 목차에 따라 구성해주세요.

        # 부동산 종합 리스크 분석 보고서 (Knock AI)

        ## 1. 종합 진단 결과
        - 최종 리스크 등급('{prediction}')과 그 의미를 요약 설명해주세요.
        - 계약 시 가장 주의해야 할 핵심 포인트를 짚어주세요.

        ## 2. 상세 위험 요소 분석
        - 내부 분석 자료를 바탕으로 각 위험 요소가 무엇이며 왜 위험한지 상세히 설명해주세요.
        - 최신 웹 정보를 근거로 들어 설명의 전문성을 더해주세요.

        ## 3. 안전 계약을 위한 Action Plan
        - 앞으로 무엇을 해야 하는지, 구체적인 행동 방안을 단계별로 제안해주세요.

        ### [내부 분석 자료]
        {analysis}
        """
    )
    chain = prompt | llm | StrOutputParser()
    final_report = chain.invoke({
        "analysis": state['initial_analysis'],
        "prediction": prediction
    })
    return {"final_report": final_report}

# #### 3-3. 그래프 구성 및 컴파일
workflow = StateGraph(ReportState)
workflow.add_node("generate_context_query", generate_context_query)
workflow.add_node("retrieve_context", retrieve_context)
workflow.add_node("generate_initial_analysis", generate_initial_analysis)
workflow.add_node("generate_final_report", generate_final_report)
workflow.set_entry_point("generate_context_query")
workflow.add_edge("generate_context_query", "retrieve_context")
workflow.add_edge("retrieve_context", "generate_initial_analysis")
workflow.add_edge("generate_initial_analysis", "generate_final_report")
workflow.add_edge("generate_final_report", END)
app = workflow.compile()
print("\n✅ 3단계: LangGraph 워크플로우 컴파일 완료")


# ### 4. 워크플로우 실행 및 최종 리포트 확인
initial_state = {"input_data": initial_input}
final_state = app.invoke(initial_state)

print("\n" + "="*50)
print("🎉 최종 생성된 전문가 리포트 🎉")
print("="*50)
print(final_state.get("final_report"))

✅ 1단계: 입력 데이터 준비 완료
{
  "features": {
    "건축물_유형": "아파트",
    "근저당권_개수": 1,
    "채권최고액": 600000000,
    "신탁_등기여부": false,
    "압류_가압류_개수": 1,
    "선순위_채권_존재여부": true,
    "전입_가능여부": true,
    "우선변제권_여부": true,
    "주소": "서울특별시 강남구 테헤란로 123",
    "과거_전세가율": "81.50%"
  },
  "model_prediction": {
    "prediction": "주의",
    "risk_probability": "36.59%",
    "risk_score": "0.2087"
  }
}
👉 생성된 검색 쿼리: ['전세가율 높은 빌라 위험성', '등기부등본 근저당권 위험성', '가압류 있는 집 전세 계약', '전세 계약시 주의사항 최신']
👉 수집된 URL (0개): []...
⚠️ 검색된 URL이 없어 RAG 프로세스를 중단합니다.
❌ 2단계: Vector DB 구축 실패.
❌ 2단계: Retriever가 없어 Multi-Query Retriever를 설정할 수 없습니다.

✅ 3단계: LangGraph 워크플로우 컴파일 완료
--- 1. RAG 검색용 질문 생성 노드 ---
👉 생성된 RAG 질문: 이 데이터를 기반으로 한 핵심 위험 요소는 다음과 같습니다:

1. **근저당권 및 채권최고액**: 근저당권이 1개 존재하며, 채권최고액이 6억 원으로 설정되어 있습니다. 이는 해당 부동산에 대한 채무 부담이 있을 수 있음을 나타냅니다.

2. **압류 및 가압류**: 압류 또는 가압류가 1건 존재합니다. 이는 해당 부동산이 법적 분쟁에 휘말려 있을 가능성을 시사합니다.

3. **선순위 채권**: 선순위 채권이 존재하여, 후순위 채권자나 임차인의 권리가 제한될 수 있습니다.

4. **전세가율**: 과거 전세가율이 81.50%로, 비교적 높은 수준입니다. 이는 전세